# COVID-19 Classification Model
---

## 1. Import Dependencies

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D , BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint ,CSVLogger
from tensorflow.keras.optimizers import Adam
from utils import load_data , visualize_samples ,visualize_accuracy_loss
import importlib
import Shared_vars    
importlib.reload(Shared_vars)  
from Shared_vars import DATA_DIR

---

## 2. Load Training and Validation Data

In [ ]:
train_data , val_data =load_data(DATA_DIR)

---

## 3. Visualize Training Samples

In [ ]:
visualize_samples(train_data)

---

## 4. Initialize ResNet50 Base Model

In [ ]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = True  # Unfreeze the base

---

## 5. Add Custom Classification Head

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

# First dense block
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)

# Second dense block
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

# Output layer
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

---

## 6. Compile the Model

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

---

## 7. Train the Model

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

checkpoint = ModelCheckpoint("/output/best_model.h5", save_best_only=True, monitor='val_accuracy')

csv_logger = CSVLogger('training_log.csv', append=False)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[early_stop, checkpoint , csv_logger]
)

---

## 8. Visualize Training Results

In [ ]:
visualize_accuracy_loss()

---